In [4]:
import pandas as pd
import numpy as np

In [5]:
#This function reads the original file, downloaded from the EIA, and calculates the top energy source in each state, as well as the percent of generation that comes from that source, for a specified year
def topFuelYear(year, output_filename):
    #Read file
    df = pd.read_csv('Net_generation_for_all_sectors_2001_2015.csv', skiprows=4)
    #Separate out state names
    df['geography'], df['fuel-type'] = df['description'].str.split(':',1).str
    #Create a column for total utility-scale generation
    df['total-util-scale'] = np.where(df['fuel-type'] == ' all fuels (utility-scale)', df[year], np.NaN)
    #Createa  column for distributed PV
    df['dist-pv'] = np.where(df['fuel-type'] == ' distributed photovoltaic', df[year], np.NaN)
    #Fill in null values and remove non-numeric values
    df['total-util-scale'].fillna(method='ffill', inplace=True)
    df['dist-pv'].fillna(method='bfill', inplace=True)
    df = df.replace('--', 0)
    df = df.dropna()
    df['total-util-scale'] = df['total-util-scale'].astype(float)
    df['dist-pv'] = df['dist-pv'].astype(float)
    df[year] = df[year].astype(float)
    #Add together utility scale and distributed PV to come up with total generation
    df['all-generation-including-dist-pv'] = df['total-util-scale'] + df['dist-pv']
    #Calculate the percent of generation that comes from each fuel type using both utility-cale and total as the denominator
    df['per-util-scale'] = df[year]/df['total-util-scale']
    df['per-util-dist'] = df[year]/df['all-generation-including-dist-pv']
    df = df[df['fuel-type'] != ' all fuels (utility-scale)']
    #Find the fuel source that has the highest percent of generation for each geography
    idx = df.groupby('geography')['per-util-scale'].idxmax()
    df2 = df.loc[idx,['geography','fuel-type','per-util-scale']]
    #Export to CSV
    df2.to_csv(output_filename)

In [6]:
topFuelYear('2015', 'top-fuel-2015.csv')
topFuelYear('2010', 'top-fuel-2010.csv')
topFuelYear('2005', 'top-fuel-2005.csv')
topFuelYear('2001', 'top-fuel-2001.csv')